In [ ]:
#flask app 
import flask
from flask_cors import CORS
from flask import jsonify
from flask import request
import matplotlib

matplotlib.use("Agg")
import json
import matplotlib.pyplot as plt
import base64
import io
import pandas as pd
import script as sp

app = flask.Flask(__name__)
CORS(app)
transformed_columns = []
obj = sp.SessionCreation()
column_dt = {}


def fetch_columns():
    global column_dt
    columns = obj.upload_file('diabetes')
    column_name_datatype = {}
    for i in columns[10:-1].split(','):
        j = i.split(':')
        column_name_datatype[j[0].strip()] = j[1].strip()
    column_dt = column_name_datatype


@app.route('/featuredata', methods=['GET'])
def getFeatureData():
    columns = list(column_dt.keys())
    dtype = list(column_dt.values())
    counter = 1
    result = []
    for i in range(len(columns)):
        obj = {"derivedFeatureId": counter, "derivedFeatureName": columns[i], "derivedFeatureDesc": dtype[i]}
        result.append(obj)
        counter += 1
    res = {'result': result}
    return jsonify(res);


@app.route('/stats', methods=['POST'])
def stats():
    data = json.loads(request.data)
    col_name = data['colName']
    obj.run_code(f"stats_df = df.describe(['{col_name}' ])")
    cur_df = obj.read_df("stats_df")
    cur_dic = cur_df.to_dict()
    dic = {}
    title = ['count', 'mean', 'std', 'min', 'max']
    values = list(cur_dic[col_name].values())
    for i in range(len(title)):
        dic[title[i]] = values[i]
    dic['status'] = 'ok'
    return dic


@app.route('/transMatplot', methods=['POST', 'GET'])
def getTransMatplotlib():
    if request.method == "GET":
        return "<h1>You're accessing using the wrong method</h1>"
    else:
        data = json.loads(request.data)
        cols = data['feature']['fName']
        cols += "_Trans"
        code = f"""import matplotlib
matplotlib.use("Agg")
import numpy as np
import matplotlib.pyplot as plt
from vzsoi.common.util import display
counts, bins = np.histogram(pdf['{cols}'])
plt.hist(pdf['{cols}'], bins, alpha=0.5, align='mid', rwidth=0.9)
plt.xlabel("Values")
plt.ylabel("Frequency")
display(plt)
                    """
        new_code = f"pdf=df.select('{cols}').toPandas()\n" + code
        res = sp.parse_output(obj.run_code(new_code))
        if res['status'] == 'ok':
            return jsonify({"img_str": 'data:image/png;base64,' + res['text'].split("'")[1]})
        else:
            return jsonify({"error": 'Error'})


@app.route('/matplotlib', methods=['POST', 'GET'])
def getMatplotlib():
    if request.method == "GET":
        return "<h1>You're accessing using the wrong method</h1>"
    else:
        data = json.loads(request.data)
        cols = data['feature']['fName']
        code = f"""import matplotlib
matplotlib.use("Agg")
import numpy as np
from vzsoi.common.util import display
import matplotlib.pyplot as plt
counts, bins = np.histogram(pdf['{cols}'])
plt.hist(pdf['{cols}'], bins, alpha=0.5, align='mid', rwidth=0.9)
plt.xlabel("Values")
plt.ylabel("Frequency")
display(plt)
                """
        new_code = f"pdf=df.select('{cols}').toPandas()\n" + code
        res = sp.parse_output(obj.run_code(new_code))
        if res['status'] == 'ok':
            return jsonify({"img_str": 'data:image/png;base64,' + res['text'].split("'")[1]})
        else:
            return jsonify({"error": 'Error while executing code ->' + res['traceback']})


@app.route('/infoDataTest', methods=['GET'])
def update_top_values_test():
    rs = obj.info_data('BloodPressure', int('10') + 1, 1)
    return jsonify({'x_values': rs['freq'].tolist(), 'y_values': rs['value'].tolist()})


@app.route('/infoData', methods=['POST'])
def update_top_values():
    data = json.loads(request.data)
    col_name = data['column']['colName']
    lim = data['limit']['limit']
    opt = data['selectedOption']['option']
    optt = 2 if opt == "Top" else 1
    rs = obj.info_data(col_name, int(lim), optt)
    return jsonify({'x_values': rs['freq'].tolist(), 'y_values': rs['value'].tolist()})


@app.route('/columns', methods=['GET'])
def getColumns():
    columns = list(column_dt.keys())
    counter = 1
    result = []
    for col in columns:
        obj = {"key": counter, "text": col,"value":col}
        result.append(obj)
        counter += 1
    res = {'result': result}
    return jsonify(res);


@app.route('/plotCode', methods=['POST'])
def plot_code():
    data = json.loads(request.data)
    code = data["code"]
    col_list = data["columns"]
    print(data,"graph")
    if len(code.strip().split('\n')) <= 3:  # just to check new lines are added for execution
        return jsonify({"error": "empty code sent for execution"})

    res = obj.runPlotCode(code,col_list)

    if res['status'] == 'ok':
        # print('start ->', res['text'][:10])
        # print('end ->', res['text'][-10:])
        return jsonify({"img_str": 'data:image/png;base64,' + res['text'].split("'")[1]})
    else:
        return jsonify({"error": 'Error while executing code ->' + res['traceback']})


@app.route('/runCode', methods=['POST'])
def run_code():
    data = json.loads(request.data)
    transformed_columns.append(f'{data["colName"][0]}_Trans')
    status = obj.transformation(data['code'], data["colName"])
    return status


@app.route('/transformedStats', methods=['POST'])
def transformed_stats():
    data = json.loads(request.data)
    col_name = data['colName']
    print("transformed",col_name)
    print("List",transformed_columns)
    if col_name in transformed_columns:
        obj.run_code(f"{col_name}_df = df.describe(['{col_name}' ])")
        trans_col = f'{col_name}_df'
        cur_df = obj.read_df(trans_col)
        cur_dic = cur_df.to_dict()
        dic = {}
        title = ['count', 'mean', 'std', 'min', 'max']
        values = list(cur_dic[col_name].values())
        for i in range(len(title)):
            dic[title[i]] = values[i]
        dic['status'] = 'ok'
        return dic
    else:
        return {'status': 'error', 'text': "No transformations are applied. "}


@app.route('/algorithm', methods=['GET', 'POST'])
def update_algorithm():
    data = json.loads(request.data)
    l = (data['data'][-1]['tableData']['id'])
    len = l + 1
    for i in range(len):
        algo_name = (data['data'][i]['algoName'])
        exp_name = str(data['data'][i]['experimentName'])
        if algo_name == "random_forest":
            algo_name = str(data['data'][i]['algoName'])
            exp_name = str(data['data'][i]['experimentName'])
            parameters = (data['data'][i]['parameters'])
            dic = json.loads(parameters)
            y_variable = str(dic['y_variable'])
            parameter1 = str(dic['criterion'])
            parameter2 = (dic['n_estimators'])
            parameter3 = (dic['max_depth'])
            parameter4 = (dic['min_sample_split'])
            parameter5 = (dic['n_jobs'])
            # print(obj.algorithm1("random_forest","ml","Outcome","gini",10,None,2,None))
            print(obj.algorithm1(algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                                 parameter5))
        elif algo_name == "logistic_regression":
            algo_name = str(data['data'][i]['algoName'])
            exp_name = str(data['data'][i]['experimentName'])
            parameters = (data['data'][i]['parameters'])
            dic = json.loads(parameters)
            y_variable = str(dic['y_variable'])
            parameter1 = str(dic['penalty'])
            parameter2 = (dic['tot'])
            parameter3 = (dic['intercept_scaling'])
            parameter4 = (dic['max_iter'])
            parameter5 = (dic['n_jobs'])
            print(algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                  parameter5)
            # print(obj.algorithm1("random_forest","ml","Outcome","gini",10,None,2,None))
            print(obj.algorithm1(algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                                 parameter5))

        else:
            algo_name = str(data['data'][i]['algoName'])
            parameters = (data['data'][i]['parameters'])
            exp_name = str(data['data'][i]['experimentName'])
            dic = json.loads(parameters)
            y_variable = str(dic['y_variable'])
            parameter1 = str(dic['objective'])
            parameter2 = (dic['colsample_bytree'])
            parameter3 = (dic['max_depth'])
            parameter4 = (dic['alpha'])
            parameter5 = (dic['n_estimators'])
            print(algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                  parameter5)

            print(obj.algorithm1(algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                                 parameter5))
    return (" ")


@app.route('/getCorrGraph', methods=['POST'])
def getCorrGraph():
    data = json.loads(request.data)
    colsList = data['colsList']
    try:
        base64Str = display(obj.runCorr(colsList))
        return jsonify({"img_str": 'data:image/png;base64,' + base64Str})
    except Exception as e:
        return jsonify({"error": 'Error while executing code ->' + str(e)})


def executeReturnImg(code):
    import numpy as np
    t = np.arange(0., 5., 0.2)
    plt.plot(t, t, 'r--', t, t ** 2, 'bs', t, t ** 3, 'g^')
    return display(plt)


def display(plt):
    # Conver to Base64 binary
    bytes = io.BytesIO()
    plt.savefig(bytes, format='png')
    bytes.seek(0)
    base64_binary = base64.b64encode(bytes.read())

    # Convert binary to String uft-8
    ENCODING = 'utf-8'
    base64_string = base64_binary.decode(ENCODING)
    return base64_string


fetch_columns()

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000, debug=False)


In [ ]:
# Livy test script 
from livy import LivySession

import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap
import numpy as np

# URL of the server which is running Livy
LIVY_URL = 'http://bkcttplna012.ebiz.verizon.com:8999'

# To make a note of active sessions.
# If there is already a session is in Idle state we can utilize that instead of creating new session
session_list = {}

# Updating the session list with current session ID and state.
res = requests.get(LIVY_URL + "/sessions").content
sessions_details = (json.loads(res))['sessions']

for i in sessions_details:
    session_list[i['id']] = i['state']
final_session_list = []
for id, state in session_list.items():
    if state == 'idle':
        final_session_list.append(id)


class SessionCreation:
    sess = LivySession
    columns = []
    FILE_NAME = ""
    pd_df = pd.DataFrame
    import_pandas = "import pandas as pd"

    def new_session(self):
        requests_session = requests.Session()
        requests_session.headers.update({"X-Requested-By": "ponba"})
        return LivySession.create(url=LIVY_URL, proxy_user='ponba',  executor_memory='4g', executor_cores=1,
                           num_executors=1, queue='computation',
                           name="Spark-test-session", check=False,requests_session=requests_session,
                           archives=['hdfs:///user/ponba/livyapp/livyenv.zip#ENV',
                                     'hdfs:///user/ponba/livyapp/algos_mlflow.zip'],
                           py_files=['hdfs:///user/ponba/livyapp/algos_mlflow.zip'],
                           spark_conf={'spark.yarn.appMasterEnv.PYSPARK_PYTHON': './ENV/livyenv/bin/python',
                                       'spark.executorEnv.PYSPARK_PYTHON': './ENV/livyenv/bin/python'})

    def resume_session(self):
        requests_session = requests.Session()
        requests_session.headers.update({"X-Requested-By": "ponba"})
        return LivySession.create(url=LIVY_URL, proxy_user='ponba', executor_memory='4g', executor_cores=1,
                           num_executors=1, queue='computation',
                           name="Spark-test-session", check=False,requests_session=requests_session,
                           archives=['hdfs:///user/ponba/livyapp/livyenv.zip#ENV',
                                     'hdfs:///user/ponba/livyapp/algos_mlflow.zip'],
                           py_files=['hdfs:///user/ponba/livyapp/algos_mlflow.zip'],
                           spark_conf={'spark.yarn.appMasterEnv.PYSPARK_PYTHON': './ENV/livyenv/bin/python',
                                       'spark.executorEnv.PYSPARK_PYTHON': './ENV/livyenv/bin/python'})

    def __init__(self):
        """
            Deciding whether there is any session available for us to utilize or we need to create a new session.
        """
        global sess;
        # LIVY_URL = 'http://10-119-32-71.ebiz.verizon.com:8999/'
        if len(final_session_list) == 0:
            sess = self.new_session()
        else:
            sess = self.resume_session()
        # Starting the new or existing session
        sess.wait()
        #import time
        #time.sleep(220)
        load_methods(sess)

        session_list[sess.session_id] = sess.state.value

    def upload_file(self, file_name):
        """
        :param file_name: Name of the csv file present in our cluster.
        :return: Nothing
        """
        sess.run(self.import_pandas)
        # self.file_name = /user/ponba/livyapp/diabetes.csv
        col = sess.run(
            f"df = spark.read.load('/user/ponba/livyapp/{file_name}.csv',format='csv', sep=',', inferSchema='true', header='true')\ndf.cache()")

        return col.text

    def get_df(self, col_name=None):
        """
        :return: Dataframe from the cluster if column value is None.
                 If column value is given returns the dataframe of specific column
        """
        if col_name is None:
            return self.pd_df
        else:
            return self.pd_df[col_name]

    def get_columns(self, file_name="diabetes"):
        """
        :param file_name: Name of the datset in the cluster.
        :return: List of all columns in the dataset.
        """
        self.upload_file(file_name)
        self.pd_df = pd.DataFrame((sess.read('df')))
        ls = list(self.pd_df.columns)
        return ls

    def read_df(self, col_name):
        return sess.read(col_name)

    def update_stats(self, col_name):
        x = self.pd_df.loc[:, col_name]
        stats_value = []

        for i in x.describe():
            stats_value.append(i)
        return stats_value

    def run_code(self, code):
        res = sess.run(code)
        return res

    def check_raise_except(self, res):
        dic = parse_output(res)
        if dic['status'] == 'ok':
            return dic['status']
        else:
            raise Exception("Error->", dic['traceback'])

    def info_data(self, col_name, lim=10, optt=1):
        res = sess.run(textwrap.dedent('''
            from vzsoi.common.util import display
            from vzsoi.common.corr import correlation_matrix

            def value_counts(spark_df=df, colm="Outcome", order=1, n=10):
                import pandas as pd
                import pyspark.sql.functions as F
                if order==1 :
                    return  pd.DataFrame(spark_df.select(colm).groupBy(colm).count().orderBy(F.asc("count")).head(n),columns=["value","freq"])
                if order==2 :
                    return  pd.DataFrame(spark_df.select(colm).groupBy(colm).count().orderBy(F.desc("count")).head(n),columns=["value","freq"])
            '''))
        self.check_raise_except(res)
        res = sess.run(f"dff = spark.createDataFrame(value_counts(df,'{col_name}',{optt},{lim}))")
        self.check_raise_except(res)
        return sess.read('dff')

    def transformation(self, code, col_name_list):
        """
        :param column_name: Name of the column without ide or any other tag
        :param code: Code to be executed on the spark session
        :return: Output of the code`
        """
        column_name = col_name_list[0]
        pre_splitted, columns_added, columns_added_string, passing_string, return_type = parse_input(code, col_name_list)
        print("coluns added ->", columns_added)
        print(passing_string, "passing")
        for i in code.splitlines():
            pre_splitted += "\t" + i + "\n"

        pre_code = f"""from pyspark.sql.functions import *
from pyspark.sql.types import *
def imputation({columns_added_string}):
{pre_splitted}
{column_name}_imputation = udf(imputation,{return_type}Type())
df = df.withColumn("{column_name}_Trans", {column_name}_imputation({passing_string}))
df.show()
                """
        xx = sess.run(pre_code)
        dic = parse_output(xx)

        if dic['status'] == 'ok':
            return dic['status']
        else:
            return dic['traceback']

    def runPlotCode(self, code,column_list):
        print("in runcode")
        #cols = code.split('\n')[2].split(':')[1].strip()  ## To get the columns listed from line # 3
        #cols = '\'' + cols.replace(",", "\',\'") + '\''
        st = ""
        print(st,"st")
        for j in column_list:
            te = f"'{j}'"
            st = st+te+","
        print(st,"stt")
        cols = st[:-1]
        print(st)
        new_code = f"pdf=df.select({cols}).toPandas()\n" + code
        res = sess.run(new_code)
        return parse_output(res)

    def algorithm1(self, algo_name, exp_name, y_variable, parameter1, parameter2, parameter3, parameter4,
                   parameter5):
        code = """from vzsoi.sklearn import utils
pd_df=df.toPandas()
x=pd_df.iloc[0:500]
y=pd_df.iloc[501:766]
from vzsoi.sklearn.{} import run
run.with_dataframe('{}',x,y,'{}','{}', {}, {}, {}, {},tracking_uri='http://10.119.36.75:5005')""".format(algo_name,
                                                                                                         exp_name,
                                                                                                         y_variable,
                                                                                                         parameter1,
                                                                                                         parameter2,
                                                                                                         parameter3,
                                                                                                         parameter4,
                                                                                                         parameter5)
        x = sess.run(code)
        return x

    def runCorr(self, colsList):
        res = sess.run(textwrap.dedent('''
        sdf = df   #add select list to filter columns
        corrDf=correlation_matrix(spark,sdf)
        '''))
        self.check_raise_except(res)
        pnadas_corr = self.read_df('corrDf')
        corr_columns = pnadas_corr['index'].tolist()
        corr = pd.DataFrame(pnadas_corr[corr_columns].values, columns=corr_columns, index=corr_columns)
        mask = np.zeros_like(corr)
        mask = np.zeros_like(corr)
        f, ax = plt.subplots(figsize=(13.5, 11.5))
        mask[np.triu_indices_from(mask)] = False
        ax = sns.heatmap(corr, mask=mask, cmap='RdBu_r', linewidth=0.5, square=True)
        return plt


def parse_input(code, column_name):
    """Helper function for transformation code.
    :param code:Code which needs to be parsed in such as way that it can be loaded into the livy session.
    :return:pre_splitted-> Str - Initial spaces are added to the code so that it fits the abstaction code.
            column_added-> List - List of all the parameters included.
            column_added_string-> All the included parameters separated by commas.
            passing_string-> All the parametrs are included with df. at the beginning.
    """
    pre_splitted = ""
    column_added = column_name
    for i in code.splitlines():
        pre_splitted += "\t" + i + "\n"
    column_added_string = ','.join(column_added)
    df_added_list = []
    for i in column_added:
        df_added_list.append('df.' + i)
    passing_string = ','.join(df_added_list)
    re_str = code.splitlines()[2]
    return_type = re_str[13:]
    return pre_splitted, column_added, column_added_string, passing_string, return_type


def parse_output(res):
    res = str(res)
    splitted = res.split(',')

    def check_error(splitted):
        msg = splitted[0].split(' ')[1][1:-2]
        return msg

    def parse_output_text(splitted):
        output = splitted[1].split('=')
        return output[1]

    def errors(splitted):
        if check_error(splitted) == 'ok':
            parse_output_text(splitted)
        else:
            return (splitted[-1])[2:-4]

    parsed_dic = {'status': check_error(splitted), 'text': parse_output_text(splitted), 'traceback': errors(splitted)}
    return parsed_dic


# Use this method to load all the default methods to be loaded to livy
# Note: This is for testing, idealy it has to be loaded as part of zip file
def load_methods(sess):
    import textwrap
    sess.run(textwrap.dedent('''
                from vzsoi.common.util import display
                from vzsoi.common.corr import correlation_matrix

                def value_counts(spark_df=df, colm="Outcome", order=1, n=10):
                    import pandas as pd
                    import pyspark.sql.functions as F
                    if order==1 :
                        return  pd.DataFrame(spark_df.select(colm).groupBy(colm).count().orderBy(F.asc(colm)).head(n),columns=["value","count"])
                    if order==2 :
                        return  pd.DataFrame(spark_df.select(colm).groupBy(colm).count().orderBy(F.desc(colm)).head(n),columns=["value","count"])
                '''))





In [ ]:
# React App JS
import React, { Component } from 'react';

import ApiService from "./actions/ApiService";
import StatsModal from './components/StatsModal';
import InfoModal from './components/InfoModal';
import GraphModal from './components/GraphModal';
import LoadingSpinner from './components/LoadingSpinner'
import AlgosModal from './components/AlgosModal'
import CodeModal from './components/CodeModal'
import TransMF from './components/TransMF'
import CorrMF from './components/CorrMF'
import 'bootstrap/dist/css/bootstrap.min.css';
import MaterialTable from "material-table";
import { Button } from 'reactstrap';
import { forwardRef } from 'react';
import AddBox from '@material-ui/icons/AddBox';
import ArrowUpward from '@material-ui/icons/ArrowUpward';
import Check from '@material-ui/icons/Check';
import ChevronLeft from '@material-ui/icons/ChevronLeft';
import ChevronRight from '@material-ui/icons/ChevronRight';
import Clear from '@material-ui/icons/Clear';
import DeleteOutline from '@material-ui/icons/DeleteOutline';
import Edit from '@material-ui/icons/Edit';
import FilterList from '@material-ui/icons/FilterList';
import FirstPage from '@material-ui/icons/FirstPage';
import LastPage from '@material-ui/icons/LastPage';
import Remove from '@material-ui/icons/Remove';
import SaveAlt from '@material-ui/icons/SaveAlt';
import Search from '@material-ui/icons/Search';
import ViewColumn from '@material-ui/icons/ViewColumn';
import CreateIcon from '@material-ui/icons/Create';
import InsertChartIcon from '@material-ui/icons/InsertChart';
import CodeIcon from '@material-ui/icons/Code';
import InfoIcon from '@material-ui/icons/Info';
import TrendingUpIcon from '@material-ui/icons/TrendingUp';
import IconButton from '@material-ui/core/IconButton';
import { FontAwesomeIcon } from '@fortawesome/react-fontawesome';
import { faSortAmountDown} from '@fortawesome/free-solid-svg-icons';
import {faEdit} from '@fortawesome/free-solid-svg-icons';
import Tooltip from '@material-ui/core/Tooltip';
//import {UnControlled as CodeMirror} from 'react-codemirror2';
import { createMuiTheme } from '@material-ui/core/styles';
import { ThemeProvider as MuiThemeProvider } from '@material-ui/core/styles'
const theme = createMuiTheme({
  palette: {
    secondary: {
      main: '#000000',
      light: '#fcfcfc',
      dark: '#000000',
    }
  },
});


const tableIcons = {
    Add: forwardRef((props, ref) => <AddBox {...props} ref={ref} />),
    Check: forwardRef((props, ref) => <Check {...props} ref={ref} />),
    Clear: forwardRef((props, ref) => <Clear {...props} ref={ref} />),
    Delete: forwardRef((props, ref) => <DeleteOutline {...props} ref={ref} />),
    DetailPanel: forwardRef((props, ref) => <ChevronRight {...props} ref={ref} />),
    Edit: forwardRef((props, ref) => <Edit {...props} ref={ref} />),
    CreateIcon: forwardRef((props, ref)=><CreateIcon {...props} ref={ref}/>),
    InsertChartIcon: forwardRef((props, ref)=><InsertChartIcon {...props} ref={ref}/>),
    CodeIcon: forwardRef((props, ref)=><CodeIcon {...props} ref={ref}/>),
    InfoIcon: forwardRef((props, ref)=><InfoIcon {...props} ref={ref}/>),
    TrendingUpIcon: forwardRef((props, ref)=><TrendingUpIcon {...props} ref={ref}/>),
    Export: forwardRef((props, ref) => <SaveAlt {...props} ref={ref} />),
    Filter: forwardRef((props, ref) => <FilterList {...props} ref={ref} />),
    FirstPage: forwardRef((props, ref) => <FirstPage {...props} ref={ref} />),
    LastPage: forwardRef((props, ref) => <LastPage {...props} ref={ref} />),
    NextPage: forwardRef((props, ref) => <ChevronRight {...props} ref={ref} />),
    PreviousPage: forwardRef((props, ref) => <ChevronLeft {...props} ref={ref} />),
    ResetSearch: forwardRef((props, ref) => <Clear {...props} ref={ref} />),
    Search: forwardRef((props, ref) => <Search {...props} ref={ref} />),
    SortArrow: forwardRef((props, ref) => <ArrowUpward {...props} ref={ref} />),
    ThirdStateCheck: forwardRef((props, ref) => <Remove {...props} ref={ref} />),
    ViewColumn: forwardRef((props, ref) => <ViewColumn {...props} ref={ref} />),
    fontawesome: forwardRef((props, ref) => <fontawesome {...props} ref={ref} />),
    faSortAmountDown: forwardRef((props,ref) => <faSortAmountDown {...props} ref={ref} />),
    faEdit: forwardRef((props,ref) => <faEdit {...props} ref={ref} />)
};

export default class App extends Component {

    constructor(props) {
        super(props);
        this.state = {
            columns: [
                { title: 'Feature Name', field: 'derivedFeatureName' },
                { title: 'Feature Data Type', field: 'derivedFeatureDesc' }
            ],
            currentColumn :null,
            featuredata: [],
            plotdata: [],
            isLoading :true,
            modal: false,
            statsModal:false,
            graphModal:false,
            algoModal:false,
            graphCol:null,
            columnNames:[],
            algocol:null,
            infoModal: false,
            codeModal:false,
            infoCol:null,
            showModal:false,
            trans:false,
            fName : null,
            fType : null,
            statsData:null,
            b64ImageStr:null,
            selectedOrders:null,
            corrModal:false,
            corrGraph:null


        }
        this.getFeatureData = this.getFeatureData.bind(this);
        this.getPlotData = this.getPlotData.bind(this);
        this.getColumns = this.getColumns.bind(this);
        
    }

    componentDidMount() {
        this.getFeatureData();
        this.getColumns(); 
        this.getCorrGraph ();      

    }

    
    getFeatureData() {
        ApiService.getFeatureData()
            .then((res) => {
                if (res.status === 200) {
                    this.setState({ featuredata: res.data.result })
                } else {
                    alert(JSON.stringify(res.data))
                }
            }).catch((error) => {
                alert(error);
            });
    }

    getPlotData(featureid) {
        ApiService.getPlotData(featureid)
            .then((res) => {
                if (res.status === 200){
                    this.setState({ plotdata: res.data })
                } else {
                    alert(JSON.stringify(res.data))
                }
            }).catch((error) => {
                alert(error);
            });
    }

    getColumns() {
        ApiService.getColumns()
            .then((res) => {
                if (res.status === 200) {
                    this.setState({ columnNames: res.data.result,isLoading:false})
                
                } else {
                    alert(JSON.stringify(res.data))
                }
            }).catch((error) => {
                alert(error);
            });
    }


    toggleStats = ()=> {
        this.setState({
            statsModal : !this.state.statsModal
        });
    }

    toggleGraph = ()=> {
        this.setState({
            graphModal : !this.state.graphModal
        });
    }


    fetchColumnData = (colId)=> {
        this.setState({currentStatsData:this.state.statsData.result.filter((item)=> item.no===colId)})
    }

    toggleInfo = ()=> {
        this.setState({
            infoModal : !this.state.infoModal
        });
    }

    toggleAlgo =()=> {
        this.setState({
            algoModal: !this.state.algoModal

        });
    }

    toggleCode= ()=> {
        this.setState({
            codeModal: !this.state.codeModal

        });
    }
    toggleTrans = ()=> {
        this.setState({
            trans : !this.state.trans
                
            });
        }

        toggleCorr= ()=> {
            this.setState({
                corrModal: !this.state.corrModal
            });
        }

    getStats(colName) {
            ApiService.getStats(colName)
                .then((res) => {
                    if (res.status === 200){
                        this.setState({ statsData: res.data,loading:!this.state.loading })                
                    } else {
                        alert(JSON.stringify(res.data))
                    }
                }).catch((error) => {
                    alert(error);
                });
        }
    
    getMatplotlibImg = (col_name) => {
        
        this.setState({fName:col_name})
        
         ApiService.getPlotImageStr("ponba",col_name,this.state.fType)
             .then((res) => {
                 if (res.status === 200) {
                     this.setState({b64ImageStr:res.data.img_str});    
                 } else {
                     alert(JSON.stringify(res.data))
                 }
             }).catch((error) => {
                 alert(error);
             });
     };

     handleSubmitAlgo=()=>{
        this.setState({algocol:this.state.columnNames});
        
        this.toggleAlgo();
    }
    getCorrGraph=() =>{

        ApiService.getCorrGraph(this.state.columnNames )
                .then((res) => {
                    if (res.status === 200){                        
                        if(res.data.error){
                            alert(res.data.error)
            
                        }else{                
                            this.setState({corrGraph: res.data.img_str});  
                            
                        }
                    } else {
                        alert(JSON.stringify(res.data))
                    }
                }).catch((error)  => {
                    alert(error);
                });
    }


    render() {
        
        return (
            
            
            
                <MuiThemeProvider theme={theme}>
                <div style={{ marginLeft: '20%', marginRight: '20%', justifyContent: 'center', alignItems: 'center' }}>
                {this.state.isLoading && <LoadingSpinner/>}
                    <MaterialTable
                        icons={tableIcons}
                        title="Features List"
                        columns={this.state.columns}
                        data={this.state.featuredata}
                        actions={[
                            {
                                position: 'row',
                                icon: ()=> <Tooltip title="Stats" placement='top'>
                                <IconButton aria-label="Stats">
                                    <TrendingUpIcon className={this.props.className}/>
                                </IconButton>
                                </Tooltip>,
                                onClick: (event, rowData) => {
                                    this.setState({currentColumn:rowData.derivedFeatureName});
                                    this.getStats(rowData.derivedFeatureName);
                                    this.toggleStats();
                                        }
                            },
                            {
                                position: 'row',
                                icon: ()=><Tooltip title="Graph" placement='top'>
                                <IconButton aria-label="Graph">
                                    <InsertChartIcon className={this.props.className}/>
                                </IconButton>

                                
                                </Tooltip>,
                                onClick: (event, rowData) => {
                                    this.getMatplotlibImg(rowData.derivedFeatureName);
                                    this.setState({graphCol:rowData.derivedFeatureName});
                                    this.toggleGraph();
                                    }

                                
                            },
                            
                            {
                                position: 'row',
                                icon: ()=> <Tooltip title="Code" placement='top'>
                                    <IconButton aria-label="Code">
                                        <CodeIcon className={this.props.className}/>
                                    </IconButton>
                                    </Tooltip>,
                                    onClick:(event,rowData) =>{
                                        this.toggleCode();
                                        this.setState({currentColumn:rowData.derivedFeatureName});
                                    }
                            },
                            {
                                position: 'row',
                                icon: ()=> <Tooltip title="Edit" placement='top'>
                                <IconButton aria-label="Edit">
                                   <FontAwesomeIcon icon={faEdit} className={this.props.className}/>
                                </IconButton>
                                </Tooltip>,
                                onClick:(event,rowData) =>{
                                    this.toggleTrans();
                                    this.setState({fName:rowData.derivedFeatureName});
                                    this.setState({fType:rowData.derivedFeatureDesc});
                                }
                                
                            },
                            {
                                position: 'row',
                                icon: ()=> <Tooltip title="Info" placement='top'>
                                <IconButton aria-label="Info">
                                    <FontAwesomeIcon icon={faSortAmountDown} className={this.props.className}/>
                                </IconButton>
                              </Tooltip>, 
                               onClick: (event, rowData) => {
                                this.setState({infoCol:rowData.derivedFeatureName});
                                this.toggleInfo();
                                }                           
                            }
                            
                        ]}
                        options={{
                            actionsColumnIndex: -1,
                            selection: true,
                            selectionProps:{
                                textcolor:'black',
                                backgroundcolor:'white',
                                align:'left'
                            },
                    //rowStyle: rowData => ({ backgroundColor: rowData.tableData.checked ? '#37b15933' : '' }),
                            headerStyle: {
                                textAlignLast: 'center',
                                padding: '16px',
                                textColor:'blue',
                                backgroundColor:'white'
                            }
                        }}
                        
                        
                    />
                    <br/>
                    <Button variant="secondary" className={this.props.className} onClick = {this.toggleCorr}>Correlation Graph</Button>{'   '}
                    {'   '}<Button variant="secondary" className={this.props.className} onClick = {this.handleSubmitAlgo} >Algorithm Selection</Button>

                    </div>
               
                <div>
                    <StatsModal show={this.state.statsModal} toggleStats = {this.toggleStats} title={this.state.currentColumn} data = {this.state.statsData} />
                </div>
                

                <div>
                    <InfoModal show={this.state.infoModal} toggleInfo = {this.toggleInfo} 
                    column = {this.state.infoCol}/>
                </div>
                <div>
                    <GraphModal show={this.state.graphModal} colName = {this.state.graphCol}
                                toggleGraph={this.toggleGraph} img_str={this.state.b64ImageStr} 
                                uid={'ponba'}  />
                </div>
                <div>
                <CodeModal show={this.state.codeModal} title={this.state.currentColumn} data={this.state.columnNames}
                             toggleCode={this.toggleCode} />
                
                </div>
                <div>
                <AlgosModal show= {this.state.algoModal} data={this.state.columnNames} toggleAlgo = {this.toggleAlgo}/>
                </div>
                <div>
                <TransMF uid={"ponba"} fName={this.state.fName} fType={this.state.ftype} data={this.state.columnNames}
                 show={this.state.trans} toggleTrans = {this.toggleTrans} title={this.state.currentColumn}  />
                </div>
                <div>
                    <CorrMF colsList={this.state.columnNames} show={this.state.corrModal} toggleCorr = {this.toggleCorr} img={this.state.corrGraph} />
                </div>
                </MuiThemeProvider>
            
        );
    }
}


In [ ]:
# API services  bridge
import axios from 'axios';

const REACT_APP_FLASK_API_URL = 'http://10-119-32-33.ebiz.verizon.com:5000';

class ApiService {
    getFeatureData() {
        return axios.get(REACT_APP_FLASK_API_URL + '/featuredata');
    }

    getColumns() {
        return axios.get(REACT_APP_FLASK_API_URL + '/columns');
    }

    getPlotData(featureid) {
        return axios.get(REACT_APP_FLASK_API_URL + '/plotdata/'+featureid);
    }
   getInfoData(colName,limit,option){
        return axios.post(REACT_APP_FLASK_API_URL + '/infoData',{column:{colName},limit:{limit},selectedOption:{option}});
   }

   codeExtract(fName,code,columns){
       if(columns==null) columns = [fName];
       console.log("Service->Fname,Col",fName,columns)
        return axios.post(REACT_APP_FLASK_API_URL + '/plotCode',{code,fName,columns});
   }
    
   runCode(colName,code){
        return axios.post(REACT_APP_FLASK_API_URL + '/runCode',{code,colName});
  }

  getStats(colName){
     return axios.post(REACT_APP_FLASK_API_URL + '/stats',{colName});
     }

   getTransformedStats(colName){
        return axios.post(REACT_APP_FLASK_API_URL + '/transformedStats',{colName});
   }

   getPlotImageStr(uid,fName,fType) {
        return axios.post(REACT_APP_FLASK_API_URL + '/matplotlib',{userId: {uid},feature:{fName},type:[fType]});
   }

   getTransPlotImageStr(uid,fName,fType) {
    return axios.post(REACT_APP_FLASK_API_URL + '/transMatplot',{userId: {uid},feature:{fName},type:[fType]});
}

   getAlgorithm(data){
     return axios.post(REACT_APP_FLASK_API_URL + '/algorithm',{"data":data});
 }

 getCorrGraph(colsList){
     return axios.post(REACT_APP_FLASK_API_URL + '/getCorrGraph',{colsList});
 }

}
export default new ApiService();
